In [4]:
import pandas as pd

In [1]:
output_path = 'data/root'
obs_header = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUEUOM']

In [ ]:
icustays = pd.read_csv('data/csvs/all_stays.csv', index_col = False)
subjects = icustays.SUBJECT_ID.unique()

In [2]:
subjects = [3, 4]

In [5]:
mimic3_path = 'MIMIC-III Clinical Database 1.4'
outputevents = pd.read_csv(os.path.join(mimic3_path, 'OUTPUTEVENTS.csv'))
labevents = pd.read_csv(os.path.join(mimic3_path, 'LABEVENTS.csv'))
if 'ICUSTAY_ID' not in labevents:
    labevents['ICUSTAY_ID'] = ''

In [8]:
s = 3
s_data = pd.read_csv(os.path.join(output_path, str(s), 'events.csv'), index_col = False)
mask_true = s_data['SUBJECT_ID']==s
s_data_true = s_data[mask_true]
print(s_data_true)
if(len(s_data_true)==len(s_data)):
    print(f'该病人不存在数据错误！ID为{s}')
else:
    print(f'病人{s} 错误数据数位： {-(len(s_data_true)-len(s_data))}')
    s_data_true.to_csv(os.path.join(output_path, str(s), 'events.csv'), index = False)

      SUBJECT_ID  HADM_ID  ICUSTAY_ID            CHARTTIME  ITEMID   VALUE  \
0              3      NaN         NaN  2101-10-12 16:07:00   50820    7.39   
1              3      NaN         NaN  2101-10-12 18:17:00   50800     ART   
2              3      NaN         NaN  2101-10-12 18:17:00   50802      -1   
3              3      NaN         NaN  2101-10-12 18:17:00   50804      22   
4              3      NaN         NaN  2101-10-12 18:17:00   50808    0.93   
...          ...      ...         ...                  ...     ...     ...   
1711           3      NaN         NaN  2101-12-20 12:00:00   51265     453   
1712           3      NaN         NaN  2101-12-20 12:00:00   51266    HIGH   
1713           3      NaN         NaN  2101-12-20 12:00:00   51267  NORMAL   
1714           3      NaN         NaN  2101-12-20 12:00:00   51268  NORMAL   
1715           3      NaN         NaN  2101-12-20 12:00:00   51274    13.2   

     VALUEUOM  
0       units  
1         NaN  
2       mEq/L  

In [9]:
count = 0 
all_count = len(subjects)

# 默认数据都不为空
flag_dict = {}
for s in subjects:
    flag_dict[s] = False

for s in subjects:
    count = count + 1
    #首先打开写好的event.csv， 清楚SUNKECT_ID 不为s的记录
    print()
    print(f'开始清除ID为{s} 的病人的错误数据！')
    s_data = pd.read_csv(os.path.join(output_path, str(s), 'events.csv'), index_col = False)
    mask_true = s_data['SUBJECT_ID']==s
    s_data_true = s_data[mask_true]
    if(len(s_data_true)==len(s_data)):
        print(f'该病人不存在数据错误！ID为{s}')
    else:
        print(f'病人{s} 错误数据数有： {-(len(s_data_true)-len(s_data))} 个')
        s_data_true.to_csv(os.path.join(output_path, str(s), 'events.csv'), index = False)
    print()
    print(f'病人{s}数据清除完毕! 现在开始存lab和out数据')
    # 写入lab和out信息
    dn = os.path.join(output_path, str(s))
    fn = os.path.join(dn, 'events.csv')
    mask_lab = labevents['SUBJECT_ID'] == s
    mask_out = outputevents['SUBJECT_ID'] == s
    lab = labevents[mask_lab][obs_header]
    out = outputevents[mask_out][obs_header]
    if s_data_true.empty: 
        flag_dict[s] = True

    if not lab.empty:
        if flag_dict[s]:
            lab.to_csv(fn, index=False)
            flag_dict[s] = False
        else:
            lab.to_csv(fn, mode='a', header=False, index=False)
    if not out.empty:
        if flag_dict[s]:
            out.to_csv(fn, index=False)
            flag_dict[s] = False
        else:
            out.to_csv(fn, mode='a', header=False, index=False)
    print(f'第{count}个 病人{s} 的events存取完毕。共 {all_count} 个病人')


开始清除ID为3 的病人的错误数据！
该病人不存在数据错误！ID为3

病人3数据清除完毕! 现在开始存lab和out数据
第1个 病人3 的events存取完毕。共 2 个病人

开始清除ID为4 的病人的错误数据！
该病人不存在数据错误！ID为4

病人4数据清除完毕! 现在开始存lab和out数据
第2个 病人4 的events存取完毕。共 2 个病人
